Ranking system for Hyper Duplex Style Alloys

In [32]:
# Imports
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [33]:
# Mo Standard
Cr_values  = [27.0, 27.5, 28.0, 28.5, 29.0]
N_values   = [0.00, 0.05, 0.10, 0.15]
C_values = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10, 0.30, 0.50, 0.70]
Mo_values = [5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0]

X_values   = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0]

In [34]:
# Input data 
X_elements = ["Ni", "Mn", "Si"]

service_temperature_C = 20

""" https://en.wikipedia.org/wiki/Prices_of_chemical_elements """
cost_usd_per_kg = {
    "Fe": 0.42,
    "C":  0.13,
    "Cr": 9.4,
    "Mo": 40.1,
    "Ni": 13.9,
    "Mn": 1.8,
    "Si": 1.7,
    "N":  0.15,
}

density_proxy = 7.8

In [35]:
# Ranking calculation proxies

def pren(Cr_wt, Mo_wt=0.0, N_wt=0.0, W_wt=0.0):
    """PREN ≈ %Cr + 3.3(%Mo + 0.5%W) + 16%N  (Si does not enter PREN)"""
    pren = Cr_wt + 3.3*(Mo_wt + 0.5*W_wt) + 16.0*N_wt
    return pren

# CHANGE FUNCTION
def strength_proxy(alloy):
    """Solid solution strengthening proxy (MPa)"""
    k = {"Cr": 35.0, "Mo": 120.0, "Ni": 20.0, "Mn": 25.0, "Si": 20.0, "C": 250.0, "N": 180.0}
    s = 0.0
    for el, wt in alloy.items():
        if el in k and wt > 0:
            s += k[el] * math.sqrt(wt)
    return s

def cost_proxy(alloy):
    """Cost proxy ($/kg)"""
    cost = 0.0
    for el, wt in alloy.items():
        el_cost = cost_usd_per_kg.get(el, 0.0)
        cost += (wt / 100.0) * el_cost
    return cost

In [36]:
# Generate data
rows = []
for X in X_elements:
    for C in C_values:
        for Cr in Cr_values:
            for N in N_values:
                for Mo in Mo_values:
                    for x_wt in X_values:
                        comp = {"C": C, "Cr": Cr, "N": N, "Mo": Mo, X: x_wt}
                        PREN = pren(Cr_wt=Cr, Mo_wt=comp.get("Mo", 0.0), N_wt=N)
                        dss  = strength_proxy(comp)

                        Fe_balance = 100.0 - (C + Cr + N + Mo + x_wt)

                        rows.append({
                            "X_element": X,
                            "C_wt%": C,
                            "Cr_wt%": Cr,
                            "N_wt%": N,
                            "Mo_wt%": Mo,
                            f"{X}_wt%": x_wt,
                            "Fe_wt% (balance)": Fe_balance,
                            "PREN": PREN,
                            "Strength_proxy_MPa": dss,
                            "Cost_proxy_$_per_kg": cost_proxy(comp),
                            "Service_T_C (info)": service_temperature_C,
                        })

df = pd.DataFrame(rows)

csv_path = "HyperDuplex_Screening1.csv"
df.to_csv(csv_path, index=False)
print(f"CSV saved: {csv_path}")
df.head(-10)

CSV saved: HyperDuplex_Screening1.csv


,X_element,C_wt%,Cr_wt%,N_wt%,Mo_wt%,Ni_wt%,Fe_wt% (balance),PREN,Strength_proxy_MPa,Cost_proxy_$_per_kg,Service_T_C (info),Mn_wt%,Si_wt%
0,Ni,0.01,27.0,0.00,5.0,0.0,67.99,43.5,475.193492,4.543013,20,NaN,NaN
1,Ni,0.01,27.0,0.00,5.0,0.5,67.49,43.5,489.335628,4.612513,20,NaN,NaN
2,Ni,0.01,27.0,0.00,5.0,1.0,66.99,43.5,495.193492,4.682013,20,NaN,NaN
3,Ni,0.01,27.0,0.00,5.0,1.5,66.49,43.5,499.688390,4.751513,20,NaN,NaN
4,Ni,0.01,27.0,0.00,5.0,2.0,65.99,43.5,503.477763,4.821013,20,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
180165,Si,0.70,29.0,0.15,10.0,NaN,57.15,64.4,881.473810,6.788135,20,NaN,3.0
180166,Si,0.70,29.0,0.15,10.0,NaN,56.65,64.4,884.249368,6.796635,20,NaN,3.5
180167,Si,0.70,29.0,0.15,10.0,NaN,56.15,64.4,886.832794,6.805135,20,NaN,4.0
180168,Si,0.70,29.0,0.15,10.0,NaN,55.65,64.4,889.259201,6.813635,20,NaN,4.5


In [ ]:
df["Score"] = df["PREN"] + df["Strength_proxy_MPa"] / 100.0
df_sorted = df.sort_values(by="Score", ascending=False)

csv_path2 = "HyperDuplex_Ranking1.csv"
df.to_csv(csv_path2, index=False)
print(f"CSV saved: {csv_path2}")
df_sorted.head(20)

,X_element,C_wt%,Cr_wt%,N_wt%,Mo_wt%,Ni_wt%,Fe_wt% (balance),PREN,Strength_proxy_MPa,Cost_proxy_$_per_kg,Service_T_C (info),Mn_wt%,Si_wt%,Score
120119,Mn,0.7,29.0,0.15,10.0,NaN,50.15,64.4,925.889736,6.917135,20,10.0,NaN,73.658897
120118,Mn,0.7,29.0,0.15,10.0,NaN,50.65,64.4,923.887969,6.908135,20,9.5,NaN,73.638880
120117,Mn,0.7,29.0,0.15,10.0,NaN,51.15,64.4,921.832794,6.899135,20,9.0,NaN,73.618328
120116,Mn,0.7,29.0,0.15,10.0,NaN,51.65,64.4,919.719693,6.890135,20,8.5,NaN,73.597197
120115,Mn,0.7,29.0,0.15,10.0,NaN,52.15,64.4,917.543472,6.881135,20,8.0,NaN,73.575435
120114,Mn,0.7,29.0,0.15,10.0,NaN,52.65,64.4,915.298114,6.872135,20,7.5,NaN,73.552981
120113,Mn,0.7,29.0,0.15,10.0,NaN,53.15,64.4,912.976577,6.863135,20,7.0,NaN,73.529766
120112,Mn,0.7,29.0,0.15,10.0,NaN,53.65,64.4,910.570538,6.854135,20,6.5,NaN,73.505705
60059,Ni,0.7,29.0,0.15,10.0,10.0,50.15,64.4,910.078348,8.127135,20,NaN,NaN,73.500783
180179,Si,0.7,29.0,0.15,10.0,NaN,50.15,64.4,910.078348,6.907135,20,NaN,10.0,73.500783


In [ ]:
# --- filtering mask (robust to float rounding) ---
mask = (
    (df["X_element"] == "Mn") &
    np.isclose(df["C_wt%"], 0.70) &
    np.isclose(df["Cr_wt%"], 29.0) &
    np.isclose(df["Mo_wt%"], 10) &
    np.isclose(df["Mn_wt%"], 9.5)
    # N_wt% left free to vary; Fe balance will follow from composition
)

# --- apply filter, (re)compute score, sort, show ---
df_fixed = df.loc[mask].copy()
df_fixed["Score"] = df_fixed["PREN"] + df_fixed["Strength_proxy_MPa"] / 100.0
df_ranked = df_fixed.sort_values(by="Score", ascending=False).reset_index(drop=True)

csv_path3 = "HyperDuplex_Filter1.csv"
df.to_csv(csv_path3, index=False)
print(f"CSV saved: {csv_path3}")
df_ranked.head(20)

,X_element,C_wt%,Cr_wt%,N_wt%,Mo_wt%,Ni_wt%,Fe_wt% (balance),PREN,Strength_proxy_MPa,Cost_proxy_$_per_kg,Service_T_C (info),Mn_wt%,Si_wt%,Score
0,Mn,0.7,29.0,0.15,10.0,NaN,50.65,64.4,923.887969,6.908135,20,9.5,NaN,73.638880
1,Mn,0.7,29.0,0.10,10.0,NaN,50.70,63.6,911.095267,6.908060,20,9.5,NaN,72.710953
2,Mn,0.7,29.0,0.05,10.0,NaN,50.75,62.8,894.423493,6.907985,20,9.5,NaN,71.744235
3,Mn,0.7,29.0,0.00,10.0,NaN,50.80,62.0,854.174269,6.907910,20,9.5,NaN,70.541743


In [ ]:
import numpy as np

# --- filter: fixed C, Cr, Mn; N = 0; Mo between 5.0 and 10.5 ---
mask = (
    (df["X_element"] == "Mn") &
    np.isclose(df["C_wt%"], 0.70) &
    np.isclose(df["Cr_wt%"], 29.0) &
    np.isclose(df["Mn_wt%"], 9.5) &
    np.isclose(df["N_wt%"], 0.0) &
    df["Mo_wt%"].between(5.0, 10.0)
)

df_mo_sweep = df.loc[mask].copy()
df_mo_sweep["Score"] = df_mo_sweep["PREN"] + df_mo_sweep["Strength_proxy_MPa"] / 100.0
df_mo_sweep = df_mo_sweep.sort_values(by="Score", ascending=False).reset_index(drop=True)

csv_path4 = "HyperDuplex_Filter2.csv"
df.to_csv(csv_path4, index=False)
print(f"CSV saved: {csv_path4}")
df_mo_sweep.head(20)


,X_element,C_wt%,Cr_wt%,N_wt%,Mo_wt%,Ni_wt%,Fe_wt% (balance),PREN,Strength_proxy_MPa,Cost_proxy_$_per_kg,Service_T_C (info),Mn_wt%,Si_wt%,Score
0,Mn,0.7,29.0,0.0,10.0,NaN,50.8,62.00,854.174269,6.90791,20,9.5,NaN,70.541743
1,Mn,0.7,29.0,0.0,9.5,NaN,51.3,60.35,844.565790,6.70741,20,9.5,NaN,68.795658
2,Mn,0.7,29.0,0.0,9.0,NaN,51.8,58.70,834.700950,6.50691,20,9.5,NaN,67.047009
3,Mn,0.7,29.0,0.0,8.5,NaN,52.3,57.05,824.558064,6.30641,20,9.5,NaN,65.295581
4,Mn,0.7,29.0,0.0,8.0,NaN,52.8,55.40,814.112205,6.10591,20,9.5,NaN,63.541122
5,Mn,0.7,29.0,0.0,7.5,NaN,53.3,53.75,803.334484,5.90541,20,9.5,NaN,61.783345
6,Mn,0.7,29.0,0.0,7.0,NaN,53.8,52.10,792.191107,5.70491,20,9.5,NaN,60.021911
7,Mn,0.7,29.0,0.0,6.5,NaN,54.3,50.45,780.642121,5.50441,20,9.5,NaN,58.256421
8,Mn,0.7,29.0,0.0,6.0,NaN,54.8,48.80,768.639719,5.30391,20,9.5,NaN,56.486397
9,Mn,0.7,29.0,0.0,5.5,NaN,55.3,47.15,756.125896,5.10341,20,9.5,NaN,54.711259


In [41]:
# --- filtering mask (robust to float rounding) ---
mask = (
    (df["X_element"] == "Mn") &
    np.isclose(df["C_wt%"], 0.70) &
    np.isclose(df["Cr_wt%"], 29.0) &
    np.isclose(df["Mo_wt%"], 6.5) &
    np.isclose(df["Mn_wt%"], 9.5)
    # N_wt% left free to vary; Fe balance will follow from composition
)

# --- apply filter, (re)compute score, sort, show ---
df_fixed = df.loc[mask].copy()
df_fixed["Score"] = df_fixed["PREN"] + df_fixed["Strength_proxy_MPa"] / 100.0
df_ranked = df_fixed.sort_values(by="Score", ascending=False).reset_index(drop=True)

csv_path5 = "HyperDuplex_Filter3.csv"
df.to_csv(csv_path5, index=False)
print(f"CSV saved: {csv_path5}")
df_ranked.head(20)

CSV saved: HyperDuplex_Filter3.csv


,X_element,C_wt%,Cr_wt%,N_wt%,Mo_wt%,Ni_wt%,Fe_wt% (balance),PREN,Strength_proxy_MPa,Cost_proxy_$_per_kg,Service_T_C (info),Mn_wt%,Si_wt%,Score
0,Mn,0.7,29.0,0.15,6.5,NaN,54.15,52.85,850.355821,5.504635,20,9.5,NaN,61.353558
1,Mn,0.7,29.0,0.10,6.5,NaN,54.20,52.05,837.563119,5.504560,20,9.5,NaN,60.425631
2,Mn,0.7,29.0,0.05,6.5,NaN,54.25,51.25,820.891344,5.504485,20,9.5,NaN,59.458913
3,Mn,0.7,29.0,0.00,6.5,NaN,54.30,50.45,780.642121,5.504410,20,9.5,NaN,58.256421


In [ ]:
# CONSIDER CPT (Critical Pitting Temperature)
# CPT = PREN x 2.5 -60 (can be -10/20 more)